In [1]:
import psycopg2, sqlalchemy as sa, requests, os, zipfile, datetime, pandas as pd

In [2]:
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
db = os.getenv('POSTGRES_DB')
server = 'localhost'

In [3]:
connection_string = f"postgresql+psycopg2://{user}:{password}@{server}/{db}"

def get_sa_engine():
    return sa.create_engine(connection_string)

In [4]:
engine = get_sa_engine()

In [12]:
cursor = engine.connect().connection.cursor()

In [6]:

agora = (datetime.datetime.now())
ano=int((agora).strftime("%Y"))
arquivo='COTAHIST_A' + str(ano)
url = 'https://bvmf.bmfbovespa.com.br/InstDados/SerHist/COTAHIST_A'+str(ano)+'.ZIP'
nome = arquivo + '.ZIP'

headers = {"Connection": "keep-alive", 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
resposta = requests.get(url, stream=True, verify=False, headers=headers)
if resposta.status_code == requests.codes.OK:
    with open(nome, 'wb') as novo_arquivo:
            for parte in resposta.iter_content(chunk_size=256):
                novo_arquivo.write(parte)
    # print("Download finalizado. Arquivo salvo como: " + nome)
    novo_arquivo.close()
else:
    resposta.raise_for_status()
    
with zipfile.ZipFile(nome , 'r') as zip_ref:
    zip_ref.extractall()

txt = pd.read_table('COTAHIST_A'+str(ano)+'.txt',names=['a','DATA','CODBDI','CODNEG','TPMERC','NOMRES','ESPECI','PREABE','PREMAX','PREMIN','PREMED','PREULT','PREOFC','PREOFV','TOTNEG','QUATOT','VOLTOT',],skiprows=1)
txt = txt.drop(txt.tail(1).index)
txt['DATA'] = txt.a.str[2:10]
txt['CODBDI'] = txt.a.str[10:12]
txt['CODNEG'] = txt.a.str[12:24]
txt['TPMERC'] = txt.a.str[24:27]
txt['NOMRES'] = txt.a.str[27:39]
txt['ESPECI'] = txt.a.str[39:49]
txt['PREABE'] = txt.a.str[56:69]
txt['PREMAX'] = txt.a.str[69:82]
txt['PREMIN'] = txt.a.str[82:95]
txt['PREMED'] = txt.a.str[95:108]
txt['PREULT'] = txt.a.str[108:121]
txt['PREOFC'] = txt.a.str[121:134]
txt['PREOFV'] = txt.a.str[134:147]
txt['TOTNEG'] = txt.a.str[147:152]
txt['QUATOT'] = txt.a.str[152:170]
txt['VOLTOT'] = txt.a.str[170:188]

txt = txt.drop('a', axis=1)

txt[['DATA']] = txt[['DATA']].applymap(lambda x: datetime.date(int(x[:4]),int(x[4:6]),int(x[6:])))
txt[['CODBDI']] = txt[['CODBDI']].applymap(lambda x: int(x))
txt[['CODNEG']] = txt[['CODNEG']].applymap(lambda x: x.replace(' ',''))
txt[['TPMERC']] = txt[['TPMERC']].applymap(lambda x: float(x))
txt[['PREABE']] = txt[['PREABE']].applymap(lambda x: float(x)/100)
txt[['PREMAX']] = txt[['PREMAX']].applymap(lambda x: float(x)/100)
txt[['PREMIN']] = txt[['PREMIN']].applymap(lambda x: float(x)/100)
txt[['PREMED']] = txt[['PREMED']].applymap(lambda x: float(x)/100)
txt[['PREULT']] = txt[['PREULT']].applymap(lambda x: float(x)/100)
txt[['PREOFC']] = txt[['PREOFC']].applymap(lambda x: float(x)/100)
txt[['PREOFV']] = txt[['PREOFV']].applymap(lambda x: float(x)/100)
txt[['TOTNEG']] = txt[['TOTNEG']].applymap(lambda x: float(x))
txt[['QUATOT']] = txt[['QUATOT']].applymap(lambda x: float(x))
txt[['VOLTOT']] = txt[['VOLTOT']].applymap(lambda x: float(x)/100)

c:\Users\bruno.dias\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bvmf.bmfbovespa.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\bruno.dias\AppData\Local\Temp\ipykernel_6784\1094670337.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt[['DATA']] = txt[['DATA']].applymap(lambda x: datetime.date(int(x[:4]),int(x[4:6]),int(x[6:])))
C:\Users\bruno.dias\AppData\Local\Temp\ipykernel_6784\1094670337.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt[['CODBDI']] = txt[['CODBDI']].applymap(lambda x: int(x))
C:\Users\bruno.dias\AppData\Local\Temp\ipykernel_6784\1094670337.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt[['CODNEG']] = txt[['CODNEG']

In [ ]:
lista_papeis = txt['CODNEG'].unique()
df_lista_papeis = pd.DataFrame(lista_papeis, columns=['ticker'])
df_lista_papeis['company_name'] = 'N/A'
df_lista_papeis.to_sql('ativos', engine, if_exists='append', index=False)

In [20]:
sql = 'select * from ativos'
cursor.execute(sql)
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=columns)

In [27]:
merged = pd.merge(txt, df, left_on='CODNEG',right_on='ticker', how='inner')
merged

,DATA,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PREABE,PREMAX,PREMIN,PREMED,PREULT,PREOFC,PREOFV,TOTNEG,QUATOT,VOLTOT,id,ticker,company_name
0,2024-01-02,2,BBDC3,10.0,BRADESCO,ON EJ N1,15.26,15.26,14.98,15.08,15.11,15.08,15.12,6674.0,4545600.0,6.857685e+07,2,BBDC3,BBDC3
1,2024-01-02,2,ALUP11,10.0,ALUPAR,UNT N2,31.46,31.56,30.25,30.53,30.45,30.43,30.53,4451.0,834400.0,2.547695e+07,3,ALUP11,ALUP11
2,2024-01-02,2,AMBP3,10.0,AMBIPAR,ON NM,16.17,16.29,15.51,15.83,15.58,15.58,15.59,7292.0,1379300.0,2.183819e+07,4,AMBP3,AMBP3
3,2024-01-02,2,RDOR3,10.0,REDE D OR,ON EJ NM,28.11,28.36,27.58,27.80,27.74,27.73,27.76,14625.0,4877200.0,1.356095e+08,5,RDOR3,RDOR3
4,2024-01-02,2,TELB3,10.0,TELEBRAS,ON,15.75,15.75,15.42,15.54,15.42,15.40,15.75,4.0,500.0,7.770000e+03,6,TELB3,TELB3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945203,2024-09-30,82,YDUQW900,80.0,YDUQE FM,ON NM,0.62,0.65,0.62,0.62,0.63,0.40,0.64,6.0,2000.0,1.250000e+03,124121,YDUQW900,YDUQW900
1945204,2024-09-30,82,YDUQW920,80.0,YDUQE FM,ON NM,0.73,0.73,0.71,0.71,0.72,0.00,0.00,6.0,4800.0,3.448000e+03,110562,YDUQW920,YDUQW920
1945205,2024-09-30,82,YDUQW940,80.0,YDUQE FM,ON NM,0.76,0.83,0.76,0.77,0.83,0.03,0.00,6.0,7300.0,5.647000e+03,123716,YDUQW940,YDUQW940
1945206,2024-09-30,82,YDUQX986,80.0,YDUQE,ON NM,1.20,1.20,1.20,1.20,1.20,0.01,0.00,3.0,1500.0,1.800000e+03,105973,YDUQX986,YDUQX986


In [28]:
merged = merged[['DATA', 'PREABE', 'PREMAX', 'PREMIN', 'id', 'PREOFC']]
merged.rename(columns={'PREABE':'price_open', 'PREMAX':'price_high', 'PREMIN':'price_low', 'PREOFC':'price', 'id': 'ativo_id', 'DATA': 'date'}, inplace=True)
merged.dropna(subset=['price'], inplace=True)

C:\Users\bruno.dias\AppData\Local\Temp\ipykernel_6784\1139809263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'PREABE':'price_open', 'PREMAX':'price_high', 'PREMIN':'price_low', 'PREOFC':'price', 'id': 'ativo_id', 'DATA': 'date'}, inplace=True)


In [32]:
merged.to_sql('ativos_prices', if_exists='append', con=engine, index=False, chunksize=5000)

389208